In [ ]:
import json
from collections import defaultdict
import pandas as pd

# 输入文件路径
dataset_file = 'Benchmark/license_data/license_llm/term/result/merged_terms.json'



markers = ["CAN", "MUST", "MUST NOT", "OPTIONAL", "NOT SPECIFIED"]
confusion_matrix = defaultdict(lambda: defaultdict(int))

with open(dataset_file, 'r') as f:
    data = json.load(f)

with open('/home/keqiang/Benchmark/fine_grained.json', 'r') as file:
    dataset = json.load(file)

dataset_dict = {
    (entry['license_name'], term_info['term'])
    for entry in dataset for term_info in entry['terms']
}


# 构建混淆矩阵函数
def build_confusion_matrix(data):
    for license_data in data:
        license_name = license_data["license_name"]
        
        for term_data in license_data["terms"]:
            term = term_data["term"]
            
            # 检查条款是否在数据集中
            if (license_name, term) not in dataset_dict:
                continue
            
            # 获取 deepseek 和 mistral 的标注值
            deepseek_marker = term_data["deepseek_response"].get("marker", "None")
            mistral_marker = term_data["mistral_large_response"].get("marker", "None")
            
            # 更新混淆矩阵
            confusion_matrix[deepseek_marker][mistral_marker] += 1


# 生成混淆矩阵
build_confusion_matrix(data)

# 转换为 DataFrame 以便查看和输出
confusion_df = pd.DataFrame(confusion_matrix).fillna(0).astype(int)
confusion_df = confusion_df.reindex(index=markers, columns=markers, fill_value=0)

# 输出混淆矩阵
print(confusion_df)

                CAN  MUST  MUST NOT  OPTIONAL  NOT SPECIFIED
CAN            1396    20         2         1              9
MUST              7   720         2        11             25
MUST NOT         35    30       736         0             37
OPTIONAL          0     6         3        37              4
NOT SPECIFIED    13    27        10         6            259
